In [ ]:
import sys

sys.path.insert(0, "../../src")

### Load a toy dataset

In [ ]:
import torch

from pytorch_adapt.datasets.getters import get_mnist_mnistm

# mnist is the source domain
# mnistm is the target domain
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], ".")
dataloader = torch.utils.data.DataLoader(
    datasets["train"], batch_size=32, num_workers=2
)

### Create models

In [ ]:
from pytorch_adapt.models import Classifier, Discriminator, MNISTFeatures

device = torch.device("cuda")

G = MNISTFeatures().to(device)
C = Classifier(num_classes=10, in_size=1200, h=256).to(device)
D = Discriminator(in_size=1200, h=256).to(device)
models = {"G": G, "C": C, "D": D}

G_opt = torch.optim.Adam(G.parameters(), lr=0.0001)
C_opt = torch.optim.Adam(C.parameters(), lr=0.0001)
D_opt = torch.optim.Adam(D.parameters(), lr=0.0001)
optimizers = [G_opt, C_opt, D_opt]

### Use in vanilla PyTorch

In [ ]:
from pytorch_adapt.hooks import DANNHook
from pytorch_adapt.utils.common_functions import batch_to_device

# Assuming that models, optimizers, and dataloader are already created.
hook = DANNHook(optimizers)
for data in dataloader:
    data = batch_to_device(data, device)
    # Optimization is done inside the hook.
    # The returned loss is for logging.
    loss, _ = hook({}, {**models, **data})
    break

### Build complex algorithms

In [ ]:
from pytorch_adapt.hooks import EntropyReducer, MeanReducer, VATHook

# G and C are the Generator and Classifier models
misc = {"combined_model": torch.nn.Sequential(G, C)}
reducer = EntropyReducer(
    apply_to=["src_domain_loss", "target_domain_loss"], default_reducer=MeanReducer()
)
hook = DANNHook(optimizers, reducer=reducer, post_g=[VATHook()])
for data in dataloader:
    data = batch_to_device(data, device)
    loss, _ = hook({}, {**models, **data, **misc})
    break

### Remove some boilerplate

In [ ]:
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models, Optimizers

# Assume G, C and D are existing models
models = Models(models)
# Override the default optimizer for G and C
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.123}), keys=["G", "C"])
adapter = DANN(models=models, optimizers=optimizers)

for data in dataloader:
    adapter.training_step(data, device)
    break

### Wrap with your favorite PyTorch framework

In [ ]:
from pytorch_adapt.frameworks import Ignite

wrapped_adapter = Ignite(adapter)
wrapped_adapter.run(datasets, epoch_length=1)

### Check accuracy of your model

In [ ]:
preds = torch.randn(10000, 100)

from pytorch_adapt.validators import SNDValidator

# Assuming predictions have been collected
target_train = {"preds": preds}
validator = SNDValidator()
score = validator.score(epoch=1, target_train=target_train)

In [ ]:
from pytorch_adapt.validators import SNDValidator

validator = SNDValidator()
wrapped_adapter.run(datasets, validator=validator, epoch_length=1)